In [161]:
import pandas as pd
import re

excel_sheet = 'BaggageFees.xlsx'

def row_contains_keyword(row, keywords):
   # Check if a row contains keywords, the keywords are the likely column names
    
    vals = [("" if pd.isna(v) else str(v).strip().lower()) for v in row.values]
    return any(word in v for word in keywords for v in vals)

def find_header_row(df):
    #Finds the row number that's probably the header
    keywords = ['rank', 'Airline', 'Airlines', '1Q','2Q','3Q','4Q','Full Year']
    for i, row in df.iterrows():
        # makes sure that the row is a header
        if row_contains_keyword(row, keywords):
            non_empty = sum(1 for v in row.values if not pd.isna(v) and str(v).strip() != '')
            if non_empty >= 2:
                return i
    return None

#Cleaning start

# Dict to store all the dataframes
all_sheets = {}
baggage = pd.ExcelFile(excel_file_path)

# 2007 to 2025
sheets = [s for s in baggage.sheet_names if re.search(r'^(2007|2008|2009|201\d|202[0-5])$', s)]

for sheet_name in sheets:
    df = pd.read_excel(baggage, sheet_name=sheet_name, header=None, dtype=object)
    header_row = find_header_row(df)
    
    # Read sheet with the detected header
    df = pd.read_excel(baggage, sheet_name=sheet_name, header=header_row, dtype=object)

    # Clean up column names
    df.columns = [("" if pd.isna(x) else str(x).strip()) for x in df.columns]

    # Find Column Airline
    col = None
    for col in df.columns:
        lowered = str(col).lower()
        if any(word in lowered for word in ['airline']):
            key_col = col
            break

    #For edge case: 2010 doesn't have airline column for some reason
    if col is None:
        all_sheets[sheet_name] = df
        continue

    df = df[df[col].notna()] # # remove rows where column is empty
    df.reset_index(drop=True, inplace=True)
    
    all_sheets[sheet_name] = df
    print(df)

   Rank                  Airline      1Q      2Q      3Q      4Q Full Year
0     1      American Airlines     28829   30015   31346   34348    124538
1     2        Delta Air Lines     20343   22447   25035   28721     96546
2     3          United Airlines   12045   13410   14829   12718     53002
3     4     Continental Airlines   10715   11045   10565   10519     42844
4     5     Northwest Airlines      8339    8956    9897   10309     37501
5     6              US Airways     5002    7526    8216    6995     27738
6     7      Southwest Airlines     5135    5508    5531    4625     20799
7     8          JetBlue Airways    3779    4011    4427    4199     16416
8     9        Alaska Airlines      3083    3969    5085    4014     16151
9    10         AirTran Airways     2106    2430    2479    2153      9168
10   11      Frontier Airlines       950    1126    1238    1304      4618
11   12      Hawaiian Airlines       899    1190    1247    1171      4505
12   13   Continental Mic

In [153]:
sheet_2025 = all_sheets['2025']
sheet_2025 = sheet_2025[sheet_2025["1Q"].notna()]
sheet_2025

,Airline,1Q,2Q,3Q,4Q,2025
0,Alaska,89836,104918,NaN,NaN,194754
1,Allegiant,104785.374,118550.654,NaN,NaN,223336.028
2,American,344245.831,379194.74,NaN,NaN,723440.571
3,Breeze,9772.315,12418.075,NaN,NaN,22190.39
4,Delta,247859,285591,NaN,NaN,533450
5,Frontier,204747.432,186578.467,NaN,NaN,391325.899
6,Hawaiian,21826.707,24760.675,NaN,NaN,46587.382
7,JetBlue,130386.912,136688.404,NaN,NaN,267075.316
8,Silver Airways,3247.295,-,NaN,NaN,3247.295
9,Southwest,21332,54508,NaN,NaN,75840


In [156]:
for sheet_name in all_sheets:
    if(sheet_name == '2025'):
        break;
    df = all_sheets[sheet_name]
    df = df.dropna() 
    all_sheets[sheet_name] = df.reset_index(drop=True)

for sheet_name in all_sheets: 
    all_sheets[sheet_name].drop('Rank', axis=1, errors='ignore', inplace=True)
    print(sheet_name)
    print(all_sheets[sheet_name])

for sheet_name, df in all_sheets.items():
    cols = [col for col in df.columns if str(col).strip().upper() in ['1Q', '2Q', '3Q', '4Q','Full Year']]
    
    for col in cols:
        df[col] = df[col].replace(['-', None], 0).astype(float)


2007
                    Airline       1Q       2Q       3Q       4Q Full Year
0       American Airlines    28829.0  30015.0  31346.0  34348.0    124538
1         Delta Air Lines    20343.0  22447.0  25035.0  28721.0     96546
2           United Airlines  12045.0  13410.0  14829.0  12718.0     53002
3      Continental Airlines  10715.0  11045.0  10565.0  10519.0     42844
4      Northwest Airlines     8339.0   8956.0   9897.0  10309.0     37501
5               US Airways    5002.0   7526.0   8216.0   6995.0     27738
6       Southwest Airlines    5135.0   5508.0   5531.0   4625.0     20799
7           JetBlue Airways   3779.0   4011.0   4427.0   4199.0     16416
8         Alaska Airlines     3083.0   3969.0   5085.0   4014.0     16151
9          AirTran Airways    2106.0   2430.0   2479.0   2153.0      9168
10      Frontier Airlines      950.0   1126.0   1238.0   1304.0      4618
11      Hawaiian Airlines      899.0   1190.0   1247.0   1171.0      4505
12   Continental Micronesia    52

In [155]:
all_sheets['2010'].rename(columns={all_sheets['2010'].columns[0]: 'Airline'}, inplace=True)
all_sheets['2010']

,Airline,1Q,2Q,3Q,4Q,2010
0,Delta,217773.0,255950.0,259473.0,219054.0,952250
1,American,128539.0,152059.0,151175.0,148890.0,580663
2,US Airways,120720.0,135601.0,131806.0,125496.0,513623
3,Continental,76603.0,91031.0,90494.0,83457.0,341585
4,United,71145.0,84824.0,83872.0,73366.0,313207
5,AirTran,35005.0,39204.0,38139.0,39800.0,152148
6,Alaska,21166.0,25394.0,34430.0,28007.0,108997
7,Spirit,16033.0,16811.0,22939.0,25720.0,81503
8,Frontier,13872.0,15470.0,14725.0,18048.0,62115
9,JetBlue,13763.0,14012.0,15494.0,13750.0,57019
